# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.



In [ ]:
!pip install ratsnlp

# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [ ]:
from ratsnlp.nlpbook.qa import QADeployArguments
args = QADeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-qa",
    max_seq_length=128,
    max_query_length=32,
)

# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

In [ ]:
import torch
from transformers import BertConfig, BertForQuestionAnswering
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
)
model = BertForQuestionAnswering(pretrained_model_config)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

인퍼런스 함수는 질문과 지문을 입력받아 토큰화를 수행한뒤,
input_ids, attention_mask, token_type_ids를 만듭니다.
이들 입력값을 파이토치 텐서 자료형으로 변환한 뒤 모델에 입력합니다.
모델 출력값은 소프트맥스 함수 적용 이전의 로짓 형태입니다.

마지막으로 모델 출력을 약간 후처리해서
정답 시작 로직의 최댓값 위치부터 정답 끝 로짓의 최대값 위치의 토큰들을 이어붙여
pred_text로 만듭니다.

In [ ]:
def inference_fn(question, context):
    if question and context:
      # 질문을 토큰화하고 인덱싱하되
      # max_query_length보다 길면 이에 맞게 자르기
        truncated_query = tokenizer.encode(
            question,
            add_special_tokens=False,
            truncation=True,
            max_length=args.max_query_length
       )
        # 앞서 처리한 질문을 지문과 함께 토큰화하고 인덱싱하되
        # 전체 길이가 max_seq_length보다 길면 지문 자르기
        inputs = tokenizer.encode_plus(
            text=truncated_query,
            text_pair=context,
            truncation="only_second",
            padding="max_length",
            max_length=args.max_seq_length,
            return_token_type_ids=True,
        )
        with torch.no_grad():
            outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})
            # 정답의 시작 위치와 관련된 로짓에서 가장 큰 값이 가리키는 토큰 위치를 알아내기
            start_pred = outputs.start_logits.argmax(dim=-1).item()
            # 정답의 끝 위치와 관련된 로짓에서 가장 큰 값이 가리키는 토큰 위치를 알아내기
            end_pred = outputs.end_logits.argmax(dim=-1).item()
            # 정답 시작부터 끝까지의 토큰들을 이어붙여 정답 만들기
            pred_text = tokenizer.decode(inputs['input_ids'][start_pred:end_pred+1])
    else:
        pred_text = ""
    return {
        'question': question,
        'context': context,
        'answer': pred_text,
    }

# 웹서비스 만들기 준비

`ngrok`은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구입니다. `ngrok`을 실행하려면 [회원가입](https://dashboard.ngrok.com/signup) 후 [로그인](https://dashboard.ngrok.com/login)을 한 뒤 [이곳](https://dashboard.ngrok.com/get-started/your-authtoken)에 접속해 인증 토큰(authtoken)을 확인해야 합니다. 예를 들어 확인된 `authtoken`이 `test111`이라면 다음과 같이 실행합니다.

```bash
!mkdir /root/.ngrok2 && echo "authtoken: test111" > /root/.ngrok2/ngrok.yml
```

In [ ]:
!mkdir /root/.ngrok2 && echo "authtoken: {이곳에 확인된 인증 토큰을 입력하세요}" > /root/.ngrok2/ngrok.yml

# 웹서비스 개시
아래처럼 실행해 인퍼런스 함수를 웹서비스로 만듭니다.

In [ ]:
from ratsnlp.nlpbook.qa import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()